<a href="https://colab.research.google.com/github/HenryLiangzy/COMP9417_Project/blob/master/Henry_s_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


In [11]:
np.random.seed(1)
TRAINING_FILE = "training.csv"
TEST_FILE = "test.csv"

topic_code = {
    'ARTS CULTURE ENTERTAINMENT': 1,
    'BIOGRAPHIES PERSONALITIES PEOPLE': 2,
    'DEFENCE': 3,
    'DOMESTIC MARKETS': 4,
    'FOREX MARKETS': 5,
    'HEALTH': 6,
    'MONEY MARKETS': 7,
    'SCIENCE AND TECHNOLOGY': 8,
    'SHARE LISTINGS': 9,
    'SPORTS': 10,
    'IRRELEVANT': 0
}

In [12]:
def preprocess(df):
    df['topic_code'] = df['topic'].apply(lambda x: topic_code[x])

    return df[['article_words', 'topic_code']]

def bag_of_word(df, model):
    vector = model
    df[model.__class__.__name__] = df['article_words'].apply(lambda x: vector.fit_transform(x))

    return df

In [13]:
# load data from file
df = pd.read_csv(TRAINING_FILE)

# pre process the y
df = preprocess(df)

# split the data
train_set, test_set = train_test_split(df, test_size=0.1)

# print(train_set)

count = CountVectorizer()

train_x = count.fit_transform(train_set['article_words'].values)
train_y = train_set['topic_code']
test_x = count.transform(test_set['article_words'].values)
test_y = test_set['topic_code']

print(train_x.shape)
print(train_y.shape[0])
print(test_x.shape)
print(test_y.shape[0])

(8550, 33913)
8550
(950, 33913)
950


In [14]:
%%time

# train
test_record = []
trianing_record = []

dtc = DecisionTreeClassifier(min_samples_leaf=2)
dtc.fit(train_x, train_y)

print("The accuracy of Training set is:", dtc.score(train_x, train_y))
print("The accuracy of Test set is:", dtc.score(test_x, test_y))
print("Training time: {} s".format(end_time-start_time))

The accuracy of Training set is: 0.9385964912280702
The accuracy of Test set is: 0.6968421052631579
Training time: 5.693456172943115 s
CPU times: user 5.42 s, sys: 42 ms, total: 5.46 s
Wall time: 5.49 s


In [15]:
predict_proba = dtc.predict_proba(test_x)
auc = roc_auc_score(test_y, predict_proba, multi_class='ovr')
print("Current auc is:", auc)

Current auc is: 0.7691544315909066
